In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from utils import *

In [5]:
data = pd.read_csv('./data/MI_timesteps.txt',sep=' ')

Part 5.1

In [ ]:
#create space


In [6]:
def delay_embed(data, delay, num_vars):
    N = len(data) - delay * num_vars
    embedded_data = np.zeros((N, delay * num_vars))
    for i in range(N):
        for j in range(num_vars):
            embedded_data[i, j*delay:(j+1)*delay] = data.iloc[i:i+delay, j]
    return embedded_data

In [7]:
# Selecting the first three measurement areas (columns 2, 3, 4)
measurements = data.iloc[:, 2:4]

# Parameters for delay embedding
num_delays = 350
delay_embedding_dim = num_delays * 3  # 350 delays * 3 measurements = 1053 dimensions
num_rows = measurements.shape[0]

# Creating the delay embeddings
delay_embeddings = []
for start_row in range(num_rows - num_delays):
    window = measurements.iloc[start_row:start_row + num_delays + 1].values.flatten()
    delay_embeddings.append(window)

# Convert the list of delay embeddings to a numpy array
delay_embeddings = np.array(delay_embeddings)

# Applying PCA
pca = PCA()
pca.fit(delay_embeddings)

# Displaying the shape of the delay embeddings and the explained variance ratio of PCA components
delay_embeddings.shape, pca.explained_variance_ratio_

((14651, 702),
 array([5.62578352e-01, 2.68229691e-01, 6.46972615e-02, 4.84950263e-02,
        2.46266734e-02, 1.36872005e-02, 9.05859052e-03, 3.93211276e-03,
        1.88299357e-03, 1.19548475e-03, 6.45539014e-04, 2.81695439e-04,
        1.44353308e-04, 1.05144508e-04, 7.64804159e-05, 6.29495994e-05,
        4.55984723e-05, 4.21908772e-05, 3.13555499e-05, 2.71063049e-05,
        2.19911934e-05, 1.22776269e-05, 8.58717463e-06, 8.07020064e-06,
        7.06422871e-06, 6.59475800e-06, 5.90370070e-06, 4.60675843e-06,
        4.23523271e-06, 3.54219652e-06, 3.52424242e-06, 3.25336720e-06,
        3.19180921e-06, 2.50018065e-06, 2.29997245e-06, 2.22391341e-06,
        1.86464622e-06, 1.61117674e-06, 1.58017570e-06, 1.37922307e-06,
        1.21779733e-06, 1.16030395e-06, 1.07745750e-06, 9.83979470e-07,
        8.96123086e-07, 8.47463253e-07, 8.47049175e-07, 8.42703722e-07,
        7.90195431e-07, 7.34362056e-07, 7.20517416e-07, 6.98301108e-07,
        6.38125587e-07, 5.82073009e-07, 5.6491114